In [0]:
#Import  packages

import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

# Set random seed for purposes of reproducibility
seed = 21

In [0]:
from keras.datasets import cifar10

# loading in the data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [0]:
# Normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# One hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]


In [0]:
# Create Model

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

# Randomly remove connections to prevent overfitting
model.add(Dropout(0.2))

# Normalize the inputs heading into the next layer
model.add(BatchNormalization())

# Another convolutional layer, but the filter size increases so the network can learn more complex representations
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))

# Another Dropout
model.add(Dropout(0.2))
# Another Normalizaton
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Flatten the data
model.add(Flatten())

model.add(Dropout(0.2))

In [0]:
# Create densly connected neural network layer
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Vote for highest probability node
model.add(Dense(class_num))
model.add(Activation('softmax'))

In [0]:
#Number of epochs and optimizer algorithm to use
epochs = 25
optimizer = 'adam'

# Compile Model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#Print Model Summary
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_14 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)       

In [0]:
#Fit the model with the seed chosen above.
numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 489s 10ms/step - loss: 1.5115 - acc: 0.4603 - val_loss: 1.1165 - val_acc: 0.5925
Epoch 2/25
50000/50000 [==============================] - 486s 10ms/step - loss: 1.0361 - acc: 0.6318 - val_loss: 0.8414 - val_acc: 0.7053
Epoch 3/25
50000/50000 [==============================] - 483s 10ms/step - loss: 0.8609 - acc: 0.6973 - val_loss: 0.7652 - val_acc: 0.7359
Epoch 4/25
50000/50000 [==============================] - 481s 10ms/step - loss: 0.7601 - acc: 0.7335 - val_loss: 0.7751 - val_acc: 0.7258
Epoch 5/25
50000/50000 [==============================] - 482s 10ms/step - loss: 0.7043 - acc: 0.7537 - val_loss: 0.6600 - val_acc: 0.7711
Epoch 6/25
50000/50000 [==============================] - 484s 10ms/step - loss: 0.6689 - acc: 0.7643 - val_loss: 0.7129 - val_acc: 0.7518
Epoch 7/25
50000/500

In [0]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 82.56%
